In [98]:
import json
import numpy as np
from sklearn.model_selection import GroupKFold, ShuffleSplit
import os
import glob
import librosa

/home/bmcfee/miniconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/bmcfee/miniconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [82]:
artist_index_simple = json.load(open('/home/bmcfee/git/milsed/models_medley/resources/medley_artist_index.json'))

In [85]:
artist_index = {}

In [93]:
for key in artist_index_simple:
    
    hits = sorted(glob.glob('/home/bmcfee/data/Medleydb_Downmix/{}*.jams'.format(key)))
    new_keys = [os.path.splitext(os.path.basename(_))[0] for _ in hits]
    
    for k in new_keys:
        artist_index[k] = artist_index_simple[key]

In [99]:
durations = {}
for key in artist_index:
    duration = librosa.get_duration(filename='/home/bmcfee/data/Medleydb_Downmix/{}.ogg'.format(key))
    durations[key] = duration

In [101]:
items = sorted(list(artist_index.keys()))

In [102]:
groups = [artist_index[item] for item in items]

In [103]:
G = GroupKFold(n_splits=5)

for fold, (train_full, test) in enumerate(G.split(X=np.arange(len(items)), groups=groups)):
    # This gives us a train-test split.
    # Now we need to make a validate split
    idx_test = [items[t] for t in test]
    S = ShuffleSplit(n_splits=1, random_state=20180227)
    for train, val in S.split(X=np.arange(len(train_full)), y=np.zeros(len(train_full))):
        idx_train = [items[train_full[t]] for t in train]
        idx_val = [items[train_full[t]] for t in val]
        
    with open('/home/bmcfee/git/milsed/models_medley/resources/index_train{:02d}.json'.format(fold), 'w') as fdesc:
        json.dump(dict(id=idx_train), fdesc, indent=2)
    with open('/home/bmcfee/git/milsed/models_medley/resources/index_validate{:02d}.json'.format(fold), 'w') as fdesc:
        json.dump(dict(id=idx_val), fdesc, indent=2)
    with open('/home/bmcfee/git/milsed/models_medley/resources/index_test{:02d}.json'.format(fold), 'w') as fdesc:
        json.dump(dict(id=idx_test), fdesc, indent=2)

In [104]:
with open('/home/bmcfee/git/milsed/models_medley/resources/durations.json', 'w') as fdesc:
    json.dump(durations, fdesc, indent=2)

---
# Create the gt annotation csv

In [107]:
import pandas as pd
import jams
from tqdm import tqdm_notebook as tqdm

In [106]:
jamses = sorted(glob.glob('/home/bmcfee/data/Medleydb_Downmix/*.jams'))

In [121]:
records = []
for jf in tqdm(jamses):
    fid = os.path.splitext(os.path.basename(jf))[0]
    jam = jams.load(jf)
    ann = jam.annotations['tag_medleydb_instruments', 0]
    
    for obs in ann:
        records.append((fid, obs.time, obs.time + obs.duration, obs.value))

HBox(children=(IntProgress(value=0, max=531), HTML(value='')))

In [126]:
df = pd.DataFrame.from_records(records, columns=['id', 'start', 'end', 'value'])

In [128]:
df.to_csv('/home/bmcfee/git/milsed/models_medley/resources/gt_all.csv', sep='\t', header=False, index=False)